In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# 데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/AI_colab/car_evaluation.csv")
df.columns = ['price', 'maint', 'doors', 'persons', 'lug_capacity','safety','output']
print(df.head())

# 데이터 확인
print("데이터 미리보기:")
print(df.head())

   price  maint doors persons lug_capacity safety output
0  vhigh  vhigh     2       2        small    med  unacc
1  vhigh  vhigh     2       2        small   high  unacc
2  vhigh  vhigh     2       2          med    low  unacc
3  vhigh  vhigh     2       2          med    med  unacc
4  vhigh  vhigh     2       2          med   high  unacc
데이터 미리보기:
   price  maint doors persons lug_capacity safety output
0  vhigh  vhigh     2       2        small    med  unacc
1  vhigh  vhigh     2       2        small   high  unacc
2  vhigh  vhigh     2       2          med    low  unacc
3  vhigh  vhigh     2       2          med    med  unacc
4  vhigh  vhigh     2       2          med   high  unacc


In [5]:
#결측치 측정기기
missing_values = df.isnull().sum()
print("각각 결측치 개수:")
print(missing_values)
print("총 결측치 개수:", df.isnull().sum().sum())

각각 결측치 개수:
price           0
maint           0
doors           0
persons         0
lug_capacity    0
safety          0
output          0
dtype: int64
총 결측치 개수: 0


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# 데이터 생성 (더 많은 데이터 추가)
data = {
    "price": ["vhigh", "vhigh", "high", "low", "med", "low", "med", "high", "vhigh", "low",
              "med", "high", "low", "med", "vhigh", "low", "high", "med", "vhigh", "low"],
    "maint": ["vhigh", "vhigh", "med", "low", "high", "med", "low", "high", "med", "high",
              "low", "med", "high", "vhigh", "med", "low", "high", "med", "vhigh", "low"],
    "doors": [2, 4, 3, 2, 5, 4, 3, 5, 4, 2, 3, 5, 2, 4, 5, 3, 2, 5, 4, 2],
    "persons": [2, 4, 4, 2, 5, 2, 4, 5, 5, 2, 4, 5, 2, 4, 5, 2, 5, 4, 5, 2],
    "lug_capacity": ["small", "med", "big", "med", "small", "big", "med", "small", "med", "big",
                     "small", "med", "big", "med", "small", "big", "med", "small", "med", "big"],
    "safety": ["med", "high", "low", "high", "med", "low", "high", "med", "low", "high",
               "med", "high", "low", "high", "med", "low", "high", "med", "low", "high"],
    "output": ["unacc", "acc", "good", "vgood", "unacc", "acc", "good", "vgood", "acc", "good",
               "unacc", "acc", "good", "vgood", "unacc", "acc", "good", "vgood", "acc", "good"]
}

# 데이터프레임 생성
df = pd.DataFrame(data)

# Label Encoding 적용 (범주형 변수 변환)
encoder = LabelEncoder()
encoded_df = df.copy()

for column in df.columns:
    if df[column].dtype == "object":
        encoded_df[column] = encoder.fit_transform(df[column])

# 특성(X)과 타겟(y) 분리
X = encoded_df.drop(columns=['output'])
y = encoded_df['output']

# 🔹 데이터 클래스 균형 유지 (test_size를 클래스 개수 이상으로 설정)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# 🚨 데이터 클래스 확인
print("훈련 데이터 클래스 분포:\n", y_train.value_counts())
print("테스트 데이터 클래스 분포:\n", y_test.value_counts())

# 클래스가 1개 이하이면 오류 발생 방지
if len(y_train.unique()) < 2:
    raise ValueError("🚨 데이터가 하나의 클래스만 포함하고 있어 학습할 수 없습니다. 다양한 클래스를 포함하도록 데이터를 수정하세요.")

# 5가지 분류 모델 정의 (KNN의 n_neighbors 수정)
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=200),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=2),  # 🔹 이웃 수 조정
    "Support Vector Machine": SVC(kernel='linear')
}

# 결과 저장
results = {}
conf_matrices = {}

# 모델 학습 및 평가
for name, model in models.items():
    model.fit(X_train, y_train)  # 모델 학습
    y_pred = model.predict(X_test)  # 예측 수행
    accuracy = accuracy_score(y_test, y_pred)  # 정확도 저장
    results[name] = accuracy
    conf_matrices[name] = confusion_matrix(y_test, y_pred)  # 혼동 행렬 저장

# 정확도 출력
print("\n모델별 정확도 비교:")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")


훈련 데이터 클래스 분포:
 output
1    4
0    4
2    2
3    2
Name: count, dtype: int64
테스트 데이터 클래스 분포:
 output
3    2
1    2
0    2
2    2
Name: count, dtype: int64

모델별 정확도 비교:
Random Forest: 0.3750
Decision Tree: 0.2500
Logistic Regression: 0.3750
K-Nearest Neighbors: 0.3750
Support Vector Machine: 0.5000
